# Exercise 08 - Size optimization for continua


## Task 1 - Book shelf

Let us consider a bookshelf that needs a support structure. The design domain is given by a unit square $x \in [0, 1]^2$ and a maximum thickness $d_{max}=0.1$. The left boundary of the domain $\partial \Omega_D$ is fixed to the wall and the top boundary $\partial \Omega_N$ is loaded with a uniform line load representing the weight of books.

<div>
    <center>
        <img src="figures/domain.png" width="250"/>
    </center>
</div>


In [ ]:
from math import sqrt
import torch
from simple_fem import FEM, export_mesh

torch.set_default_dtype(torch.double)

a) Reuse the code from last exercise´s code to reproduce the example domain in a FEM object called `square`. Use a constant thickness of $d=0.05$ for the entire domain, forces $f(\mathbf{x}) = 1.0/N, \mathbf{x} \in \partial \Omega_N$ and the material parameters $E=1000.0, \nu=0.25$.

In [ ]:
# Create nodes
N = 10

# Create elements connecting nodes

# Load at top

# Constrained displacement at left end

# Thickness

# Material

# Create and plot the domain

b) To save material, the bookshelf should use only 50% of that given design space, while being as stiff as possible to support many books. We want to achieve this by a variable thickness distribution of the component. 

You are provided with a function that performs root finding with the bisection method from a previous exercise. Now, you should implement a size optimization algorithm with MMA in a function named `optimize(fem, d_0, d_min, d_max, V_0, iter=15)` that takes the FEM model `fem`, the initial thickness distribution `d_0`, the minimum and maximum thickness distributions `d_min, d_max`, the volume constraint, and the maximum iteration count `iter` with a default value of 15.

In [ ]:
def bisection(f, a, b, max_iter=50, tol=1e-10):
    # Bisection method always finds a root, even with highly non-linear grad
    i = 0
    while (b - a) > tol:
        c = (a + b) / 2.0
        if i > max_iter:
            raise Exception(f"Bisection did not converge in {max_iter} iterations.")
        if f(a) * f(c) > 0:
            a = c
        else:
            b = c
        i += 1
    return c

In [ ]:
def optimize(fem, d_0, d_min, d_max, V_0, iter=15):
    # List for thickness results and lower asymptotes
    d = [d_0]
    L = []

    # Element-wise areas
    areas = fem.areas()
    
    # MMA parameter
    s = 0.7

    # Check if there is a feasible solution before starting iteration
    if torch.inner(d_min, areas) > V_0:
        raise Exception("d_min is not compatible with V_0.")

    # Iterate solutions
    for k in range(iter):
        # Solve the problem at d_k

        # Compute strain energy density

        # Compute lower asymptote
        if k <= 1:
            L_k = d[k] - s * (d_max - d_min)
        else:
            L_k = torch.zeros_like(L[k - 1])
            for j in range(len(L_k)):
                if (d[k][j] - d[k - 1][j]) * (d[k - 1][j] - d[k - 2][j]) < 0.0:
                    L_k[j] = d[k][j] - s * (d[k - 1][j] - L[k - 1][j])
                else:
                    L_k[j] = d[k][j] - 1 / sqrt(s) * (d[k - 1][j] - L[k - 1][j])
        L.append(L_k)

        # Compute lower move limit in this step
        d_min_k = torch.maximum(d_min, 0.9 * L[k] + 0.1 * d[k])

        # Analytical solution for d

        # Analytical gradient

        # Solve dual problem

        # Compute current optimal point with dual solution and append it to solutions

    return d

c) Now run the optimization with $d_0=0.05, d_{min}=0.001, d_{max}=0.1$ and a volume constraint $V_0=50\%$ of the maximum design volume.

In [ ]:
# Initial thickness, minimum thickness, maximum thickness

# Initial volume (50% of maximum design volume)

# Optimize and visualize results

d) Plot the evolution of design variables vs. iterations. What does the graph tell you?

In [ ]:
import matplotlib.pyplot as plt


e) How do you interpret the design? Decide which manufacturing process you would like to use and use a CAD software to create a design based on your optimization.

In [ ]:
export_mesh(square, "exercise_08_sizing.vtk", elem_data={"Thickness": [d_opt[-1]]})